In [12]:
import re, nltk, os, collections

from nltk.stem.porter import *
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

# R0672532
DOCUMENTS_DIR = "IRTM"
OUTPUT_DIR = "."
VECTORFILE_DIR = ".\Vector Files"
dictionary = {}
seq_dict = {}
all_terms = {}
for filename in os.listdir(DOCUMENTS_DIR):
    txt_dict = {}
    txt_term = []
    terms = ''
    # 開啟news.txt檔並作初步的分割、去跳行及全小寫化
    raw = open(os.path.join(DOCUMENTS_DIR, filename), 'r', encoding='UTF-8').read().lower().strip().split()

    # token化分割的單詞
    tokens = []
    for text in raw:
        tokens.append(re.compile("[^a-zA-Z]").sub('', text))

    # 使用nltk的PorterStemmer
    stemmer = PorterStemmer()
    singles = [stemmer.stem(token) for token in tokens]

    # 開啟stopwords.txt去除stopwords並輸出結果
    stopwords = open(os.path.join(OUTPUT_DIR, 'stopwords.txt'), 'r', encoding='UTF-8').read()
    filtered_words = [word for word in singles if word not in stopwords]

    #將txt file每個term整理成dictionary並計算df
    for word in filtered_words:
        if(word not in dictionary):
            dictionary.update({word: 1})
        else:
            dictionary[word] +=  1
        terms+= (word + ' ')
    od = collections.OrderedDict(sorted(dictionary.items(), key=lambda t: t[0]))
    output_file2 = open(os.path.join(OUTPUT_DIR, 'dictionary.txt'), 'w', encoding='UTF-8')
    output_file2.write('t_index\tterm\tdf\n')
    seq = 1
    #輸出所有dictionary內的term及df
    for k, v in od.items():
        seq_dict.update({k: seq})
        output_file2.write(str(seq) + '\t' + k + '\t' + str(v) + '\n')
        seq+=1
    output_file2.close()
    all_terms.update({str(filename):[terms]})
    
#使用scikit-learn計算tf-idf
for filename in os.listdir(DOCUMENTS_DIR):
    #將詞語換為矩陣
    vectorizer=CountVectorizer()
    #計算每個詞語的tf-idf
    transformer=TfidfTransformer()
    #前個fit_transform用於計算tf-idf，後個fit_transform將詞語換為矩陣
    tfidf=transformer.fit_transform(vectorizer.fit_transform(all_terms[str(filename)]))
    #獲取所有詞語
    word=vectorizer.get_feature_names()
    #將tf-idf轉為矩陣
    weight=tfidf.toarray()
    output_file3 = open(os.path.join(VECTORFILE_DIR, filename), 'w', encoding='UTF-8')
    output_file3.write(str(len(word)) + '\nt_index\ttf-idf\n')
    #輸出結果
    for i in range(len(weight)):
        for j in range(len(word)):  
            if(str(word[j]) in seq_dict):
                output_file3.write(str(seq_dict[str(word[j])]) + '\t' + str(weight[i][j]) + '\n')
            else:
                output_file3.write('ERROR\n')
    output_file3.close()
                
#讀取docname的vector file並轉為矩陣
def read_file_to_list(docname):
    doc_content = open(os.path.join(VECTORFILE_DIR, docname), 'r', encoding='UTF-8').readlines()
    doc_content = doc_content[2:]
    result = []
    for line in doc_content:
        result.append(line.replace('\n', '').split('\t'))
    return result

#計算cosine similarity並print出結果
def cosine(docx, docy):
    docx_list = read_file_to_list(docx)
    docy_list = read_file_to_list(docy)
    cosine_sim_score = 0.0
    
    i, j = 0, 0
    
    while (i < len(docx_list)) and (j < len(docy_list)):
        x_t_index = int(docx_list[i][0])
        y_t_index = int(docy_list[j][0])
        x_tfidf = float(docx_list[i][1])
        y_tfidf = float(docy_list[i][1])
        
        if x_t_index == y_t_index:
            cosine_sim_score += x_tfidf * y_tfidf
            i += 1
            j += 1
        else:
            if x_t_index > y_t_index:
                j += 1
            else:
                i += 1
    print(cosine_sim_score)

cosine('1.txt', '2.txt')

0.2415868251107653
